Possible Errors	Reason: 

Missing PET	        In PxListA but not in the folderA
Not matching PET	In the FolderA but not in the PxListB
Missing CT	        In PxListB but not in the folderB
Not Matching RT	    In the csv file2 but empty
Dicom2nii error	    In the csvfile3 file but not in the csvfile4

PxListTot = /Users/Luis/CSVs/PxListTot  - Full List of Px
PxListA =  Px with CT and RT collected and Matched - from csv file 2
PxListB =  Px with PET and ACCT collected and Matched

folderA = /DICOM_data/DATA_VOLLEDIG_structured/PET_DICOM
folderB = /DICOM_data/DATA_VOLLEDIG_unstructured
csvfile1 = /Users/Luis/CSVs/PET_MatchACCT_DICOM_Paths.csv
csvfile2 = /Users/Luis/CSVs/CTandRTMatch.csv
csvfile3 = /Users/Luis/CSVs/MergedCT_RT_PET_ACCT.csv
csvfile4 = /Users/Luis/CSVs/CompleteNii_ACCT_PET_CT_RTStruct.csv

In [1]:
import os
import pandas as pd
import csv
rootPanama = "//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/"
rootOneDrive = "C:/Users/delaoArevaLR/OneDrive - UMCG/Code/Code_From_Umcg/UMCG_ImageFormattingCode/Dicom2Nifti_Public/CSVs_PanamaCode_Backup/"

In [2]:
PxListTot = pd.read_csv(rootPanama+"/Users/Luis/CSVs/PxListTot.txt",sep='\t')
print((len(PxListTot)))

873


In [3]:
#Create PxListA and NotMatchingRT
sourceCsv = os.path.join(rootOneDrive,"CTandRTMatch.csv")

PxListA =[]
NotMatchingRT = []

df = pd.read_csv(sourceCsv)
id_list = df['ID'].tolist()
CTpath_list = df['CT'].tolist()
print("Px on List A",len(id_list))

for i in range(len(id_list)):
    if len(CTpath_list[i])>2: #valid CT path
        PxListA.append(id_list[i])
    else: #Not a valid CT path
        NotMatchingRT.append(id_list[i])
        
print("Len PxListA",len(PxListA))
print("Len NotMatchingRT",len(NotMatchingRT))

Px on List A 834
Len PxListA 669
Len NotMatchingRT 165


In [4]:
#Create PxListB
sourceCsv = os.path.join(rootOneDrive,"PET_MatchACCT_DICOM_Paths.csv")
df = pd.read_csv(sourceCsv)
PxListB = df['Px'].tolist()
print("Len PxListB",len(PxListB))

Len PxListB 323


In [5]:
#Create Missing PET & Not matching PET
folderA = rootPanama+"/DICOM_data/DATA_VOLLEDIG_structured/PET_DICOM"
PxInFolderA = os.listdir(folderA)
PxInFolderA = [int(element) for element in PxInFolderA]
print("Len Px in folder A",len(PxInFolderA))
MissingPET = [patient for patient in PxListA if patient not in PxInFolderA]

print("Missing PET",len(MissingPET))
NotMatchingPET = [patient for patient in PxInFolderA if patient not in PxListB]
print("Not Matching PET",len(NotMatchingPET))

Len Px in folder A 339
Missing PET 376
Not Matching PET 22


In [6]:
# Create Missing CT -  In PxListB but not in the folderB
folderB = rootPanama+"/DICOM_data/DATA_VOLLEDIG_unstructured"
PxInFolderB = os.listdir(folderB)
PxInFolderB = [int(element) for element in PxInFolderB]
MissingCT = [patient for patient in PxListB if patient not in PxInFolderB]
print("Len Px in folder B",len(PxInFolderB))
print("Missing CT",len(MissingCT))

Len Px in folder B 834
Missing CT 0


In [7]:
#Px with Dicom2Nii errors:
csvfile3 = rootPanama+"/Users/Luis/CSVs/MergedCT_RT_PET_ACCT.csv"
csvfile4 = rootPanama+"/Users/Luis/CSVs/CompleteNii_ACCT_PET_CT_RTStruct.csv"
df = pd.read_csv(csvfile3)
PxinCsv3 = df['Px'].tolist()
df = pd.read_csv(csvfile4)
PxinCsv4 = df['ID'].tolist()
PxwDicom2NiiErr = [patient for patient in PxinCsv3 if patient not in PxinCsv4]
print("Px with Dicom2Nii Errs",len(PxwDicom2NiiErr))

Px with Dicom2Nii Errs 57


In [8]:
#MERGE all errors:
errors_dict = {}
for id in NotMatchingRT:
    errors_dict[id] = "Not Matching RT"
for id in MissingPET:
    errors_dict[id] = "Missing PET"
for id in NotMatchingPET:
    errors_dict[id] = "Not Matching PET"
for id in MissingCT:
    errors_dict[id] = "Missing CT"
for id in PxwDicom2NiiErr:
    errors_dict[id] = "Dicom2Nii Err"
    
print("Len",len(errors_dict))

Len 604


In [9]:
#SAVE List of errors it as CSV
df = pd.DataFrame(list(errors_dict.items()), columns=['ID', 'Reason'])
csv_file_path = rootOneDrive+'ListOfErrors.csv'
df.to_csv(csv_file_path, index=False)